In [ ]:
# General Object Detection + Omar Car Classification
# - Uses general YOLO model to detect all objects in images
# - For car detections, crops and classifies if it's "omar_car" using fine-tuned model

from pathlib import Path
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ultralytics import YOLO

%matplotlib inline

# Paths
PROJECT_ROOT = Path(os.getcwd()).parent
DATA_TEST = PROJECT_ROOT / 'data' / 'test'
GENERAL_MODEL = 'yolov8n.pt'  # General object detection (COCO classes)
OMAR_CAR_MODEL_PATH = PROJECT_ROOT / 'runs' / 'detect' / 'omar_car_training' / 'weights' / 'best.pt'

# Fallback to latest training run if specific path doesn't exist
if not OMAR_CAR_MODEL_PATH.exists():
    # Check in project root runs
    runs_dir = PROJECT_ROOT / 'runs' / 'detect'
    if runs_dir.exists():
        train_dirs = sorted([d for d in runs_dir.glob('*') if d.is_dir()], key=lambda x: x.stat().st_mtime, reverse=True)
        for train_dir in train_dirs:
            candidate = train_dir / 'weights' / 'best.pt'
            if candidate.exists():
                OMAR_CAR_MODEL_PATH = candidate
                print(f"Using model from: {OMAR_CAR_MODEL_PATH}")
                break
    
    # Also check in notebooks/runs
    if not OMAR_CAR_MODEL_PATH.exists():
        notebooks_runs = PROJECT_ROOT / 'notebooks' / 'runs' / 'detect'
        if notebooks_runs.exists():
            train_dirs = sorted([d for d in notebooks_runs.glob('*') if d.is_dir()], key=lambda x: x.stat().st_mtime, reverse=True)
            for train_dir in train_dirs:
                candidate = train_dir / 'weights' / 'best.pt'
                if candidate.exists():
                    OMAR_CAR_MODEL_PATH = candidate
                    print(f"Using model from: {OMAR_CAR_MODEL_PATH}")
                    break

print(f"Test folder: {DATA_TEST.exists()}")
print(f"Omar car model: {OMAR_CAR_MODEL_PATH.exists()}")

# Load models
print("\n📦 Loading models...")
general_model = YOLO(GENERAL_MODEL)  # General object detection
if OMAR_CAR_MODEL_PATH.exists():
    omar_car_model = YOLO(str(OMAR_CAR_MODEL_PATH))
    print("✅ Loaded fine-tuned omar_car model")
else:
    omar_car_model = None
    print("⚠️  Omar car model not found, will only do general detection")

# COCO class names (for general detection)
COCO_CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
    'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat',
    'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
    'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Car-related classes in COCO (class ID 2 is 'car')
CAR_CLASS_IDS = [2, 5, 7]  # car, bus, truck


In [ ]:
# Helper function to crop image based on bounding box
def crop_image(img, box, padding=10):
    """Crop image using bounding box coordinates with padding."""
    x1, y1, x2, y2 = box
    h, w = img.shape[:2]
    x1 = max(0, int(x1) - padding)
    y1 = max(0, int(y1) - padding)
    x2 = min(w, int(x2) + padding)
    y2 = min(h, int(y2) + padding)
    return img[y1:y2, x1:x2]

# Process single image
def process_image(img_path, general_model, omar_car_model, conf_threshold=0.25):
    """Process image: detect all objects, then classify cars."""
    # Load image
    img = np.array(Image.open(img_path).convert('RGB'))
    original_img = img.copy()
    
    # General object detection
    results = general_model.predict(
        source=img,
        imgsz=640,
        conf=conf_threshold,
        device=0 if torch.cuda.is_available() else 'cpu',
        verbose=False
    )
    
    detections = []
    car_crops = []
    car_boxes = []
    
    if len(results) > 0 and results[0].boxes is not None:
        boxes = results[0].boxes
        for i in range(len(boxes)):
            cls = int(boxes.cls[i])
            conf = float(boxes.conf[i])
            box = boxes.xyxy[i].cpu().numpy()
            class_name = COCO_CLASSES[cls] if cls < len(COCO_CLASSES) else f"class_{cls}"
            
            detections.append({
                'class': class_name,
                'class_id': cls,
                'confidence': conf,
                'box': box,
                'is_car': cls in CAR_CLASS_IDS
            })
            
            # If it's a car, crop and classify with omar_car model
            if cls in CAR_CLASS_IDS and omar_car_model is not None:
                car_crop = crop_image(img, box, padding=20)
                if car_crop.size > 0:
                    car_crops.append(car_crop)
                    car_boxes.append((box, conf, class_name))
    
    # Classify car crops
    car_classifications = []
    if len(car_crops) > 0 and omar_car_model is not None:
        car_results = omar_car_model.predict(
            source=car_crops,
            imgsz=640,
            conf=0.3,
            device=0 if torch.cuda.is_available() else 'cpu',
            verbose=False
        )
        for car_result in car_results:
            is_omar_car = False
            omar_car_conf = 0.0
            if car_result.boxes is not None and len(car_result.boxes) > 0:
                # Check if omar_car was detected
                for j in range(len(car_result.boxes)):
                    cls = int(car_result.boxes.cls[j])
                    conf = float(car_result.boxes.conf[j])
                    if cls == 0:  # omar_car is class 0
                        is_omar_car = True
                        omar_car_conf = max(omar_car_conf, conf)
            car_classifications.append({
                'is_omar_car': is_omar_car,
                'confidence': omar_car_conf
            })
    
    return {
        'image': original_img,
        'detections': detections,
        'car_boxes': car_boxes,
        'car_classifications': car_classifications
    }

print("✅ Helper functions defined")


In [ ]:
# Visualize results with different colors for general objects vs omar_car
def visualize_detections(result, img_path):
    """Visualize detections with color coding."""
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    img = result['image']
    ax.imshow(img)
    
    car_idx = 0
    for det in result['detections']:
        box = det['box']
        x1, y1, x2, y2 = box
        w, h = x2 - x1, y2 - y1
        
        if det['is_car']:
            # Car detection - check if it's omar_car
            if car_idx < len(result['car_classifications']):
                car_class = result['car_classifications'][car_idx]
                if car_class['is_omar_car']:
                    # Green for omar_car
                    color = 'lime'
                    label = f"🚗 omar_car ({car_class['confidence']:.2f})"
                    linewidth = 3
                else:
                    # Yellow for other cars
                    color = 'yellow'
                    label = f"🚙 {det['class']} (other car)"
                    linewidth = 2
                car_idx += 1
            else:
                color = 'yellow'
                label = f"🚙 {det['class']}"
                linewidth = 2
        else:
            # Other objects - blue
            color = 'cyan'
            label = f"{det['class']} ({det['confidence']:.2f})"
            linewidth = 1.5
        
        # Draw bounding box
        rect = patches.Rectangle(
            (x1, y1), w, h,
            linewidth=linewidth,
            edgecolor=color,
            facecolor='none'
        )
        ax.add_patch(rect)
        
        # Add label
        ax.text(
            x1, y1 - 5,
            label,
            color='white',
            fontsize=10,
            bbox=dict(facecolor=color, alpha=0.7, pad=2),
            weight='bold'
        )
    
    ax.set_title(f"Detection Results: {Path(img_path).name}", fontsize=14, weight='bold')
    ax.axis('off')
    plt.tight_layout()
    return fig

print("✅ Visualization function defined")


In [ ]:
# Process all test images
assert DATA_TEST.exists(), f"Test folder not found: {DATA_TEST}"

# Get all image files
IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.webp', '.avif'}
test_images = [f for f in DATA_TEST.iterdir() if f.suffix.lower() in IMAGE_EXTS]
test_images = sorted(test_images)

print(f"Found {len(test_images)} test images")
print(f"Processing on: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Process each image
all_results = []
for idx, img_path in enumerate(test_images, 1):
    print(f"\n[{idx}/{len(test_images)}] Processing: {img_path.name}")
    try:
        result = process_image(img_path, general_model, omar_car_model, conf_threshold=0.25)
        result['image_path'] = str(img_path)
        all_results.append(result)
        
        # Print summary
        num_objects = len(result['detections'])
        num_cars = sum(1 for d in result['detections'] if d['is_car'])
        num_omar_cars = sum(1 for cc in result['car_classifications'] if cc['is_omar_car'])
        print(f"   Detected: {num_objects} objects ({num_cars} cars, {num_omar_cars} omar_car)")
    except Exception as e:
        print(f"   ❌ Error processing {img_path.name}: {e}")

print(f"\n✅ Processed {len(all_results)} images successfully")


In [ ]:
# Visualize all results
print(f"Visualizing {len(all_results)} images...\n")

for result in all_results:
    img_path = result['image_path']
    fig = visualize_detections(result, img_path)
    plt.show()
    plt.close(fig)


In [ ]:
# Summary statistics
print("=" * 60)
print("DETECTION SUMMARY")
print("=" * 60)

total_objects = 0
total_cars = 0
total_omar_cars = 0
object_counts = {}

for result in all_results:
    total_objects += len(result['detections'])
    for det in result['detections']:
        class_name = det['class']
        object_counts[class_name] = object_counts.get(class_name, 0) + 1
        if det['is_car']:
            total_cars += 1
    
    total_omar_cars += sum(1 for cc in result['car_classifications'] if cc['is_omar_car'])

print(f"\nTotal images processed: {len(all_results)}")
print(f"Total objects detected: {total_objects}")
print(f"Total cars detected: {total_cars}")
print(f"Total omar_car detected: {total_omar_cars}")
print(f"\nTop detected objects:")
for obj, count in sorted(object_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  {obj}: {count}")

print("\n" + "=" * 60)
print("Legend:")
print("  🟢 Lime box = omar_car")
print("  🟡 Yellow box = other car")
print("  🔵 Cyan box = other object")
print("=" * 60)
